# Sudoku

Our layout should look something like this:
`
    boxes = 
        ['A1', 'A2', 'A3', ...]
`

In [1]:
def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]


digits = '123456789'
rows = 'ABCDEFGHI'
cols = digits
boxes = cross(rows, cols)

| r | r | r | r | r | r | r | r | r |
|---|---|---|---|---|---|---|---|---|
| c | s | s |   |   |   |   |   |   |
| c | s | s |   |   |   |   |   |   |
| c |   |   |   |   |   |   |   |   |
| c |   |   |   |   |   |   |   |   |
| c |   |   |   |   |   |   |   |   |
| c |   |   |   |   |   |   |   |   |
| c |   |   |   |   |   |   |   |   |
| c |   |   |   |   |   |   |   |   |

#### row_units
Element example:

`row_units[0] = ['A1', 'A2', 'A3', 'A4', ...]`

This is the top most row.


#### column_units
Element example:

`column_units[0] = ['A1', 'B1', 'C1', ...]`

This is the left most column


#### square_units
Element example:

`square_units[0] = ['A1', 'A2', 'A3', 'B1', ...]`

This the top left square


In [2]:
row_units = [cross(r, cols) for r in rows]

column_units = [cross(rows, c) for c in cols]

square_units = [
    cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI')
    for cs in ('123', '456', '789')
    ]

unitlist = row_units + column_units + square_units

##### grid_values(grid)
A function to convert the string representation of a puzzle into a dictionary form.
Where `keys` are the boxes and `values` are all the available values for the box

`..3.2.6..9 ...`
into
`
   {
     'A1': '12',
     'A2': '6789',
     'A3': '3'
   }

`

In [4]:
def grid_values(grid):
    "Covert grid string into {<box>: <value>} dict with '123456789' value for empties"
    values = []
    all_digits = '123456789'
    for c in grid:
        if c =='.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

### Strategy 1: Elimination
If a box has a value assigned, then none of the **peers** of this box can have this value

| 7 | %7 | %7 | %7 | %7 | %7 | %7 | %7 | %7 |
|---|---|---|---|---|---|---|---|---|
| %7 |   |   |   |   |   |   |   |   |
| %7 |   |   |   |   |   |   |   |   |
| %7 |   |   |   |   |   |   |   |   |
| %7 |   |   |   |   |   |   |   |   |
| %7 |   |   |   |   |   |   |   |   |
| %7 |   |   |   |   |   |   |   |   |
| %7 |   |   |   |   |   |   |   |   |
| %7 |   |   |   |   |   |   |   |   |

##### eliminate()
takes a puzzle in dictionary form as input. 
The function iterates over all the boxes in the puzzle that only have one value assigned to them, and it will remove this value from every of its peers.

In [5]:
def eliminate(values):
    """
        Go through all the boxes, and whenever there is a box with a single
        value, eliminate this value form the set of values of all its peers
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit, '')
    return values

### Strategy 2: Only Choice
If there is only one box in a unit which would allow a certain digit, then that box mus be assigned that digit.

| 49 | 2 | 147 |
|---|---|---|
| 3 | 47 | 5 |
| 8 | 79 | 6 |

| 49 | 2 | 1 |
|---|---|---|
| 3 | 47 | 5 |
| 8 | 79 | 6 |


In [6]:
def only_choice(values):
    """
        Go through all the units, and whenever there is a unit with a
        value that only fits in one box, assign the value to this box.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

### Applying Contraint Propgagation to Sudoku
Constraints:

* Elimination
* Only Choice

##### reduce_puzzle
receives the unsolved puzzle as input and applies our two constraints repeatedly in an attempt to solve it.

In [8]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        
        # Eliminate strategy
        values = eliminate(values)
        # Only-choice strategy
        values = only_choice(values)
        
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # if no values where added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values
        

## Harder Sudokus
The current implementation has problems solving harder sudokus.
We'll use a third strategy, Search to solve that.

### Strategy 3: Search
Pick a box with a minimal number of possible values. Try to solve each of the puzzles obtained by choosing each of these values, recursively.

Using **Depth First Search**

In [9]:
def search(values):
    """Using depth-first search and propagation,
        create a search tree and solve the sudoku
    """
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes):
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest posibilities
    n, s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt